In [1]:
%matplotlib qt
%autosave 10
    
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from PIL import Image
from roifile import ImagejRoi
from skimage.draw import polygon


Autosaving every 10 seconds


In [7]:
NEEDS_ROTATE = {
    'T1_22.12.06.tif',
}

In [8]:
# Path to the large TIFF image
base_path = Path('/home/cedric/coral')
base_path = Path('/media/cedric/Storage1/coral_data')
image_path = base_path / 'T1_22.12.06.tif'
image_path = "/media/cedric/Storage1/coral_data/T1_23.02.03.tif"
base_path = Path("/media/cedric/Storage1/coral_data/T1_23.02.03")
image_path = base_path / "T1_23.02.03.tif"
# Load the image using tifffile
full_image = tifffile.imread(image_path)
if image_path.name in NEEDS_ROTATE:
    full_image = cv2.rotate(full_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
print(f"Image shape: {full_image.shape}")

h, w, _ = full_image.shape


Image shape: (14602, 14847, 4)


In [9]:
def create_mask_from_roi(roi_files, image_shape):
    # Initialize mask as a 2D array
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    height, width = image_shape[:2]

    for roi_file in roi_files:
        roi = ImagejRoi.fromfile(roi_file)
        
        # Get the coordinates of the ROI
        coords = roi.coordinates()
        if coords is None or len(coords) == 0:
            continue  # Skip if no coordinates
        
        # Convert coordinates to NumPy array
        coords = np.array(coords, dtype=np.intp)
        x_coords = coords[:, 0]
        y_coords = coords[:, 1]
        
        # Ensure coordinates are within image bounds
        if x_coords.max() >= width or y_coords.max() >= height:
            print(f"Coordinates exceed image dimensions in {roi_file}")
            continue  # Skip or adjust the coordinates
        
        # Fill the polygon in the mask
        rr, cc = polygon(y_coords, x_coords, shape=mask.shape)
        mask[rr, cc] = 1  # Assign a label value (e.g., 1)
    
    return mask


In [10]:
# Directory containing ROI files
roi_dir = base_path / 'roi/'

# List of ROI files
roi_files = sorted(roi_dir.glob('*.roi'))

# Create the mask
full_mask = create_mask_from_roi(roi_files, full_image.shape)
print("Mask created successfully!")


Mask created successfully!


In [14]:
def create_and_save_tiles(image, tile_size, output_dir):
    h, w, _ = image.shape
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    count = 0
    for i in range(0, h, tile_size):
        for j in range(0, w, tile_size):
            # Extract tile from image and mask
            tile_img = image[i:i+tile_size, j:j+tile_size]

            # Skip tiles that are smaller than the desired size
            if tile_img.shape[:2] != (tile_size, tile_size):
                continue

            # use cv2 to save lossless as png   
            # convert to uint8 bgr
            tile_img = cv2.cvtColor(tile_img, cv2.COLOR_RGBA2BGRA).astype(np.uint8)
            cv2.imwrite(output_dir / f'{count:05d}.png', tile_img)
            count += 1

    print(f"Tiles saved to {output_dir}")

# Parameters
tile_size = 1024  # Size of each tile (NxN)
# output_directory = base_path / 'dataset' / 'T1_22.12.06'
output_directory = base_path / 'test_dataset2' / 'train' / 'T1_23.02.03'


# replace the 4th channel with the mask
full_image[:, :, 3] = (1-full_mask) * 255

# Create and save tiles
create_and_save_tiles(full_image, tile_size, output_directory)


Tiles saved to /media/cedric/Storage1/coral_data/T1_23.02.03/test_dataset2/train/T1_23.02.03


In [11]:
def draw_alpha_as_contours(tile_img):
    # note tile_img is RGBA
    # Ensure the image is contiguous and in the correct format
    tile_img = np.ascontiguousarray(tile_img)
    
    # Extract alpha channel
    alpha_channel = 255 - tile_img[:, :, 3]
    
    # Find contours in the alpha channel
    contours, hierarchy = cv2.findContours(alpha_channel, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours as outline hollow
    tile_img = tile_img[:, :, :3].copy()
    cv2.drawContours(tile_img, contours, -1, (255, 255, 255), 1)
    
    return tile_img

# Load and process the tile
# tile_img = Image.open(output_directory / '00042.png')
# tile_img = Image.open(output_directory / '00131.png')
images = list(output_directory.glob('*.png'))
for image_path in images:
    tile_img = Image.open(image_path)
    tile_img = np.array(tile_img).astype(np.uint8)
    alpha = 255 - tile_img[:, :, 3]
    if alpha.max() > 0:
        tile_img = draw_alpha_as_contours(tile_img)

        # Visualize the tile with the outlined alpha channel
        plt.figure(figsize=(10, 10))
        plt.imshow(tile_img)
        plt.axis('off')
        plt.title(f'{image_path.name} with Outlined Alpha Channel')
        plt.show()



In [29]:
start_row = int(3/4*h)
end_row = -1
start_col = 0
end_col = int(1/4*w)

image = full_image[start_row:end_row, start_col:end_col]
mask = full_mask[start_row:end_row, start_col:end_col]

In [31]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'image' and 'mask' are already defined
# 'image' shape: (height, width, channels)
# 'mask' shape: (height, width), values 0 or 1

# Convert the mask to uint8 type with values 0 and 255
mask_uint8 = (mask * 255).astype(np.uint8)

# Find contours in the mask
contours, hierarchy = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the image to draw contours on
image_with_contours = image.copy()

# Define the contour color (e.g., blue) and thickness
contour_color = (0, 0, 255)  # BGR format for OpenCV (blue color)
thickness = 2  # Thickness of the contour lines

# Draw contours on the image
cv2.drawContours(image_with_contours, contours, -1, contour_color, thickness)

# Display the result
plt.figure(figsize=(12, 12))
plt.imshow(image_with_contours)
plt.title('Image with Mask Outlines')
plt.axis('off')
plt.show()
